## All the packages you need, working out-of-the-box

#### Qiskit

In [131]:
from qiskit import QuantumCircuit, Aer
import numpy as np

In [50]:
theta = [np.pi/4, np.pi/2, 3*np.pi/4, np.pi]

In [51]:
qskt_circuit = QuantumCircuit(2)

qskt_circuit.h([0, 1])
qskt_circuit.ry(theta[0], 0)
qskt_circuit.rz(theta[1], 1)
qskt_circuit.ry(theta[2], 0)
qskt_circuit.rz(theta[3], 1)

qskt_circuit.draw()

┌───┐┌─────────┐┌──────────┐
q_0: ┤ H ├┤ Ry(π/4) ├┤ Ry(3π/4) ├
     ├───┤├─────────┤└┬───────┬─┘
q_1: ┤ H ├┤ Rz(π/2) ├─┤ Rz(π) ├──
     └───┘└─────────┘ └───────┘

In [119]:
# qskt_sim = Aer.get_backend('statevector_simulator')
# qskt_sim_job = qskt_sim.run(qskt_circuit)
# qskt_sim_result = qskt_sim_job.result()
# qskt_sim_result.get_counts()

#### Braket

In [54]:
from braket import circuits as bk_circuits
from braket import devices as bk_devices

In [55]:
bk_circuit = bk_circuits.Circuit()

theta = [np.pi/4, np.pi/2, 3*np.pi/4, np.pi]

bk_circuit.h([0, 1])
bk_circuit.ry(0, theta[0])
bk_circuit.rz(1, theta[1])
bk_circuit.ry(0, theta[2])
bk_circuit.rz(1, theta[3])

print(bk_circuit)

T  : |0|    1    |   2    |
                           
q0 : -H-Ry(0.785)-Ry(2.36)-
                           
q1 : -H-Rz(1.57)--Rz(3.14)-

T  : |0|    1    |   2    |


In [120]:
# bk_sim = bk_devices.LocalSimulator(backend="braket_sv")
# bk_sim_job = bk_sim.run(bk_circuit, shots=1024)
# bk_sim_result = bk_sim_job.result()
# bk_sim_result.measurement_counts

## What devices are available to me?

#### Qiskit

In [121]:
from qiskit import IBMQ

In [80]:
# IBMQ.load_account()

In [122]:
qskt_backends = []
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
for backend in provider.backends():
    print(backend)

ibmq_qasm_simulator
ibmq_armonk
ibmq_santiago
ibmq_bogota
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_manila


In [123]:
for backend in Aer.backends():
    print(backend)

aer_simulator
aer_simulator_statevector
aer_simulator_density_matrix
aer_simulator_stabilizer
aer_simulator_matrix_product_state
aer_simulator_extended_stabilizer
aer_simulator_unitary
aer_simulator_superop
qasm_simulator
statevector_simulator
unitary_simulator
pulse_simulator


#### Braket

In [124]:
import boto3

In [125]:
client = boto3.client('braket')

In [126]:
client.search_devices(filters=[])

{'ResponseMetadata': {'RequestId': 'bc021c0b-d7cb-4f06-bc56-732e328f149c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 28 Jan 2022 23:56:02 GMT',
   'content-type': 'application/json',
   'content-length': '25091',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bc021c0b-d7cb-4f06-bc56-732e328f149c',
   'access-control-allow-origin': '*',
   'strict-transport-security': 'max-age=63072000; includeSubDomains; preload',
   'access-control-allow-headers': '*,authorization,date,x-amz-date,x-amz-security-token,x-amz-target,content-type,x-amz-content-sha256,x-amz-user-agent,x-amzn-platform-id,x-amzn-trace-id',
   'x-amz-apigw-id': 'MrlW1FhZoAMFpLQ=',
   'access-control-allow-methods': 'OPTIONS,GET,PUT,POST,DELETE',
   'access-control-expose-headers': 'x-amzn-errortype,x-amzn-requestid,x-amzn-errormessage,x-amzn-trace-id,x-amz-apigw-id,date',
   'x-amzn-trace-id': 'Root=1-61f48291-095a164b5e3145706bf85ada'},
  'RetryAttempts': 0},
 'devices': [{'deviceArn': 'arn:aws:braket:

## qBraid-SDK

In [132]:
from qbraid import get_devices

In [134]:
get_devices()

Provider,Name,qBraid ID,Status
AWS,Braket Default Simulator,aws_braket_default_sim,●
AWS,Density Matrix Simulator,aws_dm_sim,●
AWS,State Vector Simulator,aws_sv_sim,●
AWS,Tensor Network Simulator,aws_tn_sim,●
D-Wave,Advantage_system4,aws_dwave_advantage_system4,●
D-Wave,DW_2000Q_6,aws_dwave_2000Q_6,●
Google,Bristlecone,google_bristlecone,○
Google,Cirq Density Matrix Simulator,google_cirq_dm_sim,●
Google,Cirq Sparse Simulator,google_cirq_sparse_sim,●
Google,Foxtail,google_foxtail,○


In [135]:
get_devices({"paradigm": "gate-based", "type": "QPU", "qubits": {"$gte": 5}, "status": "ONLINE"})

Provider,Name,qBraid ID,Status
IBM,IBMQ Belem,ibm_q_belem,●
IBM,IBMQ Lima,ibm_q_lima,●
IBM,IBMQ Quito,ibm_q_quito,●
IBM,IBMQ Santiago,ibm_q_manila,●
IonQ,IonQ Device,aws_ionq,●
Rigetti,Aspen-11,aws_rigetti_aspen_11,●


In [101]:
bk_unitary = bk_circuits.unitary_calculation.calculate_unitary(bk_circuit.qubit_count, bk_circuit.instructions)

In [102]:
bk_sim_result = bk_dm.run(bk_circuit, shots=1024).result()

In [163]:
bk_sim_result.measurements
# bk_sim_result.measurement_counts
# bk_sim_result.measurement_probabilities

array([[0, 0],
       [1, 0],
       [0, 0],
       ...,
       [1, 1],
       [1, 1],
       [0, 1]])

#### Cirq

In [98]:
import cirq

In [111]:
cirq_sim = cirq.DensityMatrixSimulator()

In [142]:
qs = cirq.LineQubit.range(2)
# qs.reverse()

cirq_circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.H(q1),
    cirq.ry(rads=theta[0])(qs[0]),
    cirq.rz(rads=theta[1])(qs[1]),
    cirq.ry(rads=theta[2])(qs[0]),
    cirq.rz(rads=theta[3])(qs[1]),
    cirq.measure(q0, key=str(qs[0].x)),
    cirq.measure(q1, key=str(qs[1].x)),
)

print(cirq_circuit)

0: ───H───Ry(0.25π)───Ry(0.75π)───M───

1: ───H───Rz(0.5π)────Rz(π)───────M───


In [127]:
cirq_unitary = cirq_circuit.unitary()

In [156]:
cirq_sim_result = cirq_sim.run(cirq_circuit, repetitions=1024)

In [164]:
cirq_sim_result.measurements
# cirq_sim_result.data
# cirq_sim_result.histogram
# cirq_sim_result.multi_measurement_histogram(keys=cirq_sim_result.measurements.keys())

{'0': array([[0],
        [0],
        [1],
        ...,
        [0],
        [0],
        [1]], dtype=int8),
 '1': array([[1],
        [1],
        [0],
        ...,
        [1],
        [1],
        [0]], dtype=int8)}

In [129]:
cirq_sim_meas = cirq_sim_result.measurements

In [130]:
print(cirq_sim_meas)

{'0': array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int8), '1': array([[1],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]], dtype=int8)}


In [11]:
from braket.circuits import Circuit as BkCircuit
from braket.devices import LocalSimulator as BkSimulator
from braket.aws import AwsDevice

import numpy as np

In [44]:
aws_dm1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")

In [45]:
# Please enter the S3 bucket you created during onboarding in the code below

my_bucket = os.environ["s3_bucket"]
my_prefix = os.environ["s3_prefix"]

# my_bucket = "amazon-braket-Your-Bucket-Name" # the name of the bucket
# my_prefix = "Your-Folder-Name" # the name of the folder in the bucket

s3_folder = (my_bucket, my_prefix)

In [46]:
aws_dm1_job = aws_dm_sim.run(bk_circuit, s3_folder, shots=1024)

In [47]:
aws_dm1_job.metadata()["status" ]

'CREATED'

In [48]:
aws_dm1_result = job.result()

In [49]:
aws_dm1_counts = aws_dm1_result.measurement_counts

In [58]:
print(aws_dm1_counts) # <class 'collections.Counter'>

Counter({'1011': 138, '0011': 134, '1110': 128, '0110': 127, '1111': 127, '0111': 124, '1010': 124, '0010': 122})


In [24]:
import qbraid

In [26]:
aws_device = device_wrapper("aws_dm_sim")

In [29]:
aws_device._s3_location

('amazon-braket-592242689881', 'simulator-output')

In [30]:
import os
 = 'amazon-braket-592242689881'
 = 'simulator-output'

In [ ]:
from qbraid import (
    get_devices, 
    refresh_devices, 
    random_circuit, 
    device_wrapper, 
)

Get a complete list of devices available through qBraid using the get_devices function

In [ ]:
get_devices()

Filter your device search with many different advanced query options! 

For example, let's find all gate-based QPUs with at least 5 qubits that are online:

In [ ]:
get_devices({"paradigm": "gate-based", "type": "QPU", "qubits": {"$gte": 5}, "status": "ONLINE"})

This time, let's search for state vector simulators available through AWS or IBM

In [ ]:
get_devices({"type": "Simulator", "name": {"$regex": "Density Matrix"}, "vendor": {"$in": ["AWS", "Google"]}})

The qBraid SDK makes running quantum circuits on any device extremely easy. 

Simply copy the desired qBraid ID into the device wrapper function and your on your way

In [ ]:
aws_device = device_wrapper("aws_dm_sim")

In [ ]:
google_device = device_wrapper("google_cirq_dm_sim")

In [ ]:
ibm_device = device_wrapper("ibm_q_lima") 

In [ ]:
ibm_device.info

In [ ]:
circuit = random_circuit("qiskit", num_qubits=3, depth=5, measure=True)
print(circuit)

In [ ]:
shots=1000

In [ ]:
aws_job = aws_device.run(circuit, shots=shots)

In [ ]:
google_job = google_device.run(circuit, shots=shots)

In [ ]:
ibm_job = ibm_device.run(circuit, shots=shots)

In [ ]:
aws_job.id

In [ ]:
# ibm_job.device.pending_jobs()
ibm_job.status()

In [ ]:
ibm_result = ibm_job.result()

In [ ]:
google_result = google_job.result()

In [ ]:
aws_result = aws_job.result()

In [ ]:
aws_result.measurement_counts()

In [ ]:
google_result.measurement_counts()

In [ ]:
ibm_result.measurement_counts()

In [ ]:
ibm_result.plot_counts()

In [ ]:
aws_result.plot_counts()

In [ ]:
google_result.plot_counts()